# Email Service - Send to all customers

### Download dependencies

In [31]:
%pip install openai bs4 pandas requests

### Create Zero Shot prompt
This will be used to guide LLM when classifying text

In [ ]:
text = """
# Identity

You are AI model that classifies text into one of the 4 distinct categories. You will be given text as input. That is what you need to classify into one of the categories. 

The 4 categories are **Immigration**, **Economy**, **Healthcare**, **Civil Rights**

# Instructions

* Do not answer in a sentence at all. 
* Do not give responses with Markdown formatting, just return a one word answer which corresponds to one of the 4 categories mentioned
* Never answer in a sentence. 
"""

import os

text = """
# Identity
You are AI model that classifies text into one of the 4 distinct categories. You will be given text as input. That is what you need to classify into one of the categories. 
The 4 categories are **Immigration**, **Economy**, **Healthcare**, **Civil Rights**
# Instructions
* Do not answer in a sentence at all. 
* Do not give responses with Markdown formatting, just return a one word answer which corresponds to one of the 4 categories mentioned
* Never answer in a sentence. 
"""

with open('political_text_classifier.txt', 'w') as f:
    f.write(text)


### Load Zero Shot prompt
This is needed in the future (once webscraper is fully ran)

In [1]:
with open("/prompts/political_text_classifer", "r", encoding="utf-8") as file:
    instructions = file.read()

FileNotFoundError: [Errno 2] No such file or directory: '/prompts/political_text_classifer'

### Define function to run the political text classifer
This uses OpenAI SDK and GPT 4o as base LLM

In [33]:
from openai import OpenAI
client = OpenAI()

def classifyText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        instructions=instructions,
        input=input_text
    )

    return response

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

### Run webscraper to get NYC legislation
Beautifulsoup will be used to access HTML of NYC Legistar website. We will save the scrapped data in a Pandas dataframe for further processing

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

# --- Parse data rows ---
rows = []
for tr in table.find_all('tr')[1:]:  
    cells = tr.find_all('td')
    date = cells[0].get_text(strip=True)
    committee = cells[1].get_text(strip=True)
        
    # Get the agenda link (third column - find <a> within <span>)
    span = cells[2].find('span')
    agenda_link = span.find('a') if span else None
    agenda_view_aspx = agenda_link['href']
        
    
    rows.append({
            'Date': date,
            'Committee': committee,
            'Agenda Link': agenda_view_aspx
    })

scrapped_data = pd.DataFrame(rows)
print(scrapped_data)

                                                 Date   Committee  \
0                         City Council Stated Meeting  11/25/2025   
1                Committee on Women and Gender Equity  11/25/2025   
2                        Committee on General Welfare  11/25/2025   
3                                Committee on Finance  11/25/2025   
4                             Committee on Technology  11/25/2025   
5   Committee on Mental Health, Disabilities and A...  11/25/2025   
6                               Committee on Land Use  11/25/2025   
7   Committee on Governmental Operations, State & ...  11/25/2025   
8                 Committee on Civil and Human Rights  11/25/2025   
9   Subcommittee on Landmarks, Public Sitings and ...  11/25/2025   
10              Subcommittee on Zoning and Franchises  11/25/2025   
11                                 Committee on Aging  11/25/2025   
12       Committee on Rules, Privileges and Elections  11/25/2025   
13               Committee on Civi

### Convert Panda df to CSV

In [36]:
scrapped_data.to_csv("council_meetings.csv", index=False)